### 导入库

In [1]:
# 设置 nltk_data 路径环境变量
import nltk
from nltk import data
nltk.download('punkt')
data.path.append(r'../nltk_data')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
from nltk import word_tokenize #分词
from nltk.stem import WordNetLemmatizer #词干提取

from sklearn.feature_extraction.text import TfidfVectorizer #计算tf-idf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from collections import Counter #词频统计
from pylab import *
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 数据加载


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import os
#os.chdir("/content/drive/Shared drives/项目名")
os.chdir("/content/gdrive/MyDrive/ai_lab_final")

In [4]:
data = pd.read_csv('train.csv')

## 统计样本数量

In [5]:
print('样本总数：%d' % data.shape[0])

# 计算正样本数量
poslen = sum(data['gold_label']==1)

# 计算负样本数量
neglen = sum(data['gold_label']==0)
print('正样本数量：', poslen)
print('负样本数量：', neglen)

样本总数：13000
正样本数量： 1454
负样本数量： 11546


#计算词向量


In [6]:
tfidf_model = TfidfVectorizer()
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(data['sentence']).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)
tfidf_df.head()

,00,000,001,01,02,04,042,043,05,06,...,zuckerberg,zuckerman,zulily,zurich,zwaan,zydelig,zynga,zypries,zytiga,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 切分数据

In [7]:
x_train,x_test,y_train,y_test = train_test_split(
    tfidf_df, 
    data['gold_label'].values,
    test_size=0.2)

# 创建朴素贝叶斯模型

使用词向量和编码标签进行朴素贝叶斯模型建立并进行预测得到概率。我们使用 scikit-learn 库创建基本模型。然后，利用 `fit()` 拟合函数对模型进行训练。然后，可以使用训练好的模型进行预测。

In [8]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB(alpha=0.03)
nb.fit(x_train,y_train)  # 指定标签列为新创建的 `target` 列
predicted_labels = nb.predict(x_test)

In [9]:
f1=f1_score(predicted_labels, y_test)
print(f1)

0.47492625368731567
